<a href="https://colab.research.google.com/github/nyp-sit/agods/blob/main/day5/4.feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/agods/nyp_ago_logo.png" width='400'/>

# Feature Selection

In this lab, you will learn:
- different methods to do feature selection
- differences in statistical approach and machine learning approach

## Import required libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Create the Data set

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

boston = pd.read_csv('data/boston.csv', index_col=0)
X = boston.drop('MEDV', axis=1)
y = boston['MEDV']

# Split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reconstruct dataframe
X_train = pd.DataFrame(X_train, columns=scaler.get_feature_names_out())
X_test = pd.DataFrame(X_test, columns=scaler.get_feature_names_out())

# Create degree 2 polynomial features 
poly = PolynomialFeatures(degree=2, include_bias=False)

X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

X_train = pd.DataFrame(X_train, columns=poly.get_feature_names_out())
X_test = pd.DataFrame(X_test, columns=poly.get_feature_names_out())

Here we compute the $R^2$ score (for easier comparison) of linear regression for both train and test score too see if there is overfitting

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
print("Training set score: {:.2f}".format(lr.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lr.score(X_test, y_test)))
print("Number of features used: {}".format(np.sum(lr.coef_ != 0)))

We can see that the model is overfitting.  One way to fight overfitting is to use regularized model. Lasso is known to penalize the model by driving the coefficients down to 0, in a way simplify the model. This is like automatic feature selection.

Let's try using Lasso and find the best alpha by doing a grid search.

In [ ]:
## Write your code here

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso 

alphas = np.arange(0.001, 0.01, 0.001)
print('the alphas:', alphas)

param_grid = [{'alpha': alphas}]
lasso = Lasso(selection='random', max_iter=50000)
grid_cv = GridSearchCV(lasso,
             param_grid,
             cv=5, 
             scoring='neg_root_mean_squared_error',
             return_train_score=True)

grid_cv.fit(X_train, y_train)

best_estimator = grid_cv.best_estimator_
print("Training set score: {:.2f}".format(best_estimator.score(X_train, y_train)))
print("Test set score: {:.2f}".format(best_estimator.score(X_test, y_test)))
print("Number of features used: {}".format(np.sum(best_estimator.coef_ != 0)))


### END CODE HERE ###

Notice that the number of coefficients have been reduced to 81, in other words lasso do a automatic feature selection.

Let's try using scikit learn's feature selection algorithm, e.g. Recursiv Feature Elimination. For comparison, let's also restrict the number of features to 81.

In [ ]:
from sklearn.svm import SVR 
from sklearn.feature_selection import RFE

estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=81, step=1)
X_train_rfe = selector.fit_transform(X_train, y_train)
X_train_rfe = pd.DataFrame(X_train_rfe, columns=selector.get_feature_names_out())
X_test_rfe = selector.transform(X_test)
X_test_rfe = pd.DataFrame(X_test_rfe, columns=selector.get_feature_names_out())

Using the selected feature, we fit the features with a normal Linear Regression.

In [ ]:
lr_rfe = LinearRegression()
lr_rfe.fit(X_train_rfe, y_train)

print("Training set score: {:.2f}".format(lr_rfe.score(X_train_rfe, y_train)))
print("Test set score: {:.2f}".format(lr_rfe.score(X_test_rfe, y_test)))

Let's now instead using statistical method to select coefficients based on it's fitted p-values.

In [ ]:
len(y_train)

In [ ]:
import statsmodels.api as sm

model = sm.OLS(y_train.values, X_train)

# Fit your model to your training set
result = model.fit()

# Print summary statistics of the model's performance
result.summary()

We will now only use those coefficients which has p-value <= 0.05.

In [ ]:
columns = result.pvalues[result.pvalues <= 0.05].index

X_train_stat = X_train[columns]
X_test_stat = X_test[columns]

lr_stat = LinearRegression()
lr_stat.fit(X_train_stat, y_train)

print(lr_stat.score(X_train_stat, y_train))
print(lr_stat.score(X_test_stat, y_test))